<a href="https://colab.research.google.com/github/maydogan23/Spark-NLP/blob/main/Ders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import array
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout1D, Dropout, Convolution1D
from tensorflow.keras.layers import Flatten,  LSTM, GlobalMaxPooling1D

from tensorflow.keras.layers import Embedding
import numpy as np
import pandas as pd
import urllib

In [2]:
import os
from google.colab import drive

drive.mount('/content/drive')

if os.getcwd() != "/content/drive/My Drive":
  os.chdir("/content/drive/My Drive")  # My Drive dizinine geçiş

def loadFile(filepath):

    stms = []
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            stms = [line.strip() for line in file.readlines()]
    except Exception as e:
        print(f"Error reading the file: {e}")
    return stms

urlWish = '/content/drive/My Drive/dua.txt'
urlCurse = '/content/drive/My Drive/beddua.txt'

wish = loadFile(urlWish)
curse = loadFile(urlCurse)

totalWish=len(wish)
print('Toplam Dua Sayısı: ',totalWish)
totalCurse = len(curse)
print('Toplam BedDua Sayısı: ',totalCurse)


Mounted at /content/drive
Toplam Dua Sayısı:  370
Toplam BedDua Sayısı:  1211


In [3]:
def loadFile(url):
  stms =[]
  file = urllib.request.urlopen(url)

  for line in file:
    line = line.decode("utf-8")
    if(len(line)>2):
          stm =  line.strip()
          #print(stm)
          stms.append(stm)
  return stms

In [4]:
curse= curse[:totalWish]
totalCurse = len(curse)
print('toplam dua sayısı: ',totalCurse)

toplam dua sayısı:  370


In [7]:
print(wish[:5])

["Allah'tan sağlık ve huzur dilerim.", 'Sevdiklerimin mutluluğu için dua ediyorum.', 'Her gün daha fazla bilgelik ve anlayış istiyorum.', 'İç huzurumu bulmam için dua ediyorum.', 'Aileme esenlik ve mutluluk diliyorum.']


In [8]:
print(curse[:5])

['Adın bata', 'Adı batasıca.', '', 'Ağzına sapan taşı deye.', '']


In [5]:
testWish= int(totalWish* 0.1)
testCurse = int(totalCurse * 0.1)
print('test dua ', testWish)
print('test beddua', testCurse)

trainDocs= wish[:-testWish]+curse[:-testCurse]
testDocs= wish[-testWish:]+curse[-testCurse:]
print(len(trainDocs))
print(len(testDocs))

trainLabels = np.concatenate((np.ones(totalWish-testWish),np.zeros(totalCurse-testCurse)), axis=0)
testLabels = np.concatenate((np.ones(testWish),np.zeros(testCurse)), axis=0)

print(len(trainLabels))
print(len(testLabels))

test dua  37
test beddua 37
666
74
666
74


In [6]:
allDocs= trainDocs + testDocs
print(allDocs)
print(len(allDocs))

["Allah'tan sağlık ve huzur dilerim.", 'Sevdiklerimin mutluluğu için dua ediyorum.', 'Her gün daha fazla bilgelik ve anlayış istiyorum.', 'İç huzurumu bulmam için dua ediyorum.', 'Aileme esenlik ve mutluluk diliyorum.', 'Geleceğim için umut ve inanç besliyorum.', 'İşimde başarı ve gelişim diliyorum.', 'Her daim adaletli ve dürüst olmayı umuyorum.', 'Barış ve esenlik için dua ediyorum.', 'Hayatımda pozitif enerjilerin bol olmasını istiyorum.', 'Kendime ve sevdiklerime güç ve dayanma gücü diliyorum.', 'Her yeni günde mutluluk ve sağlık dilerim.', 'Zorluklar karşısında sabır ve metanet diliyorum.', 'Günlük hayatımda kolaylık ve başarı istiyorum.', 'Ruhsal huzur ve denge için dua ediyorum.', 'Kalbimde sevgi ve şefkatin yer almasını diliyorum.', 'Hayallerime ulaşmak için azim ve kararlılık istiyorum.', 'Her adımımda rehberlik ve aydınlık dilerim.', 'Kötülüklerden korunmak için dua ediyorum.', 'İçtenlikle sevdiklerimle birlikte olmayı umuyorum.', 'Yaşamımda bolluk ve bereket diliyorum.', 'Ke

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
# Tokenize our training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(allDocs)

document_count = tokenizer.document_count
vocab_size = len(tokenizer.word_index)

# Encode training data sentences into sequences
allDocs_sequences = tokenizer.texts_to_sequences(allDocs)

# Get max training sequence length
max_length = max([len(x) for x in allDocs_sequences])

# Get our training data word index
word_index = tokenizer.word_index
print("Corpus Summary")
print("Word index:", word_index)
print("document count  :", document_count)
print("vocabulary size :", vocab_size)
print("Maximum length of the statements :", max_length)

Corpus Summary
Word index: {'ve': 1, 'için': 2, 'her': 3, 'allah': 4, 'istiyorum': 5, 'diliyorum': 6, 'olsun': 7, 'hayatımda': 8, 'dua': 9, 'ediyorum': 10, 'sevgi': 11, 'kendi': 12, 'versin': 13, 'daim': 14, 'adın': 15, 'dolu': 16, 'olmasını': 17, 'gün': 18, 'umuyorum': 19, 'ağzın': 20, 'bir': 21, 'olmayı': 22, 'yeni': 23, 'bulmak': 24, 'görmeyesin': 25, 'olasın': 26, 'ah': 27, 'daha': 28, 'ola': 29, 'seni': 30, 'kurusun': 31, 'gele': 32, 'dilerim': 33, 'ruhsal': 34, 'rehberlik': 35, 'olarak': 36, 'olmak': 37, 'olmasın': 38, 'başına': 39, 'sevdiklerimin': 40, 'mutluluk': 41, 'bol': 42, 'anı': 43, 'huzur': 44, 'başarı': 45, 'destek': 46, 'ruhumun': 47, 'bereketi': 48, 'gelesin': 49, 'batsın': 50, 'kara': 51, 'umut': 52, 'sevdiklerime': 53, 'günde': 54, 'kendimi': 55, 'yaşamak': 56, 'yüzü': 57, 'taş': 58, 'elin': 59, 'ağzından': 60, 'fazla': 61, 'barış': 62, 'günlük': 63, 'sevdiklerimle': 64, 'birlikte': 65, 'eksik': 66, 'iyi': 67, 'açık': 68, 'dert': 69, 'ol': 70, 'altın': 71, 'ömrün': 

In [10]:
train_sequences = tokenizer.texts_to_sequences(trainDocs)

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=max_length)

# Output the results of our work
print("Train Doc Summary")
print("\nTraining sequences:\n", train_sequences)
print("\nPadded training sequences:\n", train_padded[:5])
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))

Train Doc Summary

Training sequences:
 [[208, 80, 1, 44, 33], [40, 209, 2, 9, 10], [3, 18, 28, 61, 144, 1, 210, 5], [211, 81, 212, 2, 9, 10], [430, 145, 1, 41, 6], [213, 2, 52, 1, 214, 82], [215, 45, 1, 431, 6], [3, 14, 216, 1, 217, 22, 19], [62, 1, 145, 2, 9, 10], [8, 218, 219, 42, 17, 5], [108, 1, 53, 83, 1, 220, 221, 6], [3, 23, 54, 41, 1, 80, 33], [222, 223, 146, 1, 224, 6], [63, 8, 225, 1, 45, 5], [34, 44, 1, 109, 2, 9, 10], [226, 11, 1, 227, 147, 228, 6], [84, 85, 2, 229, 1, 110, 5], [3, 86, 35, 1, 230, 33], [231, 232, 2, 9, 10], [111, 64, 65, 22, 19], [112, 233, 1, 148, 6], [55, 234, 2, 235, 1, 149, 5], [236, 237, 238, 2, 144, 33], [63, 239, 45, 1, 240, 19], [3, 241, 242, 21, 243, 87, 244, 6], [11, 16, 113, 114, 2, 9, 10], [245, 246, 2, 62, 1, 41, 5], [3, 43, 16, 16, 56, 2, 35, 6], [12, 150, 247, 2, 46, 6], [8, 248, 1, 249, 24, 2, 9, 10], [34, 36, 250, 1, 251, 5], [252, 36, 253, 1, 254, 22, 6], [40, 151, 1, 88, 17, 5], [255, 256, 257, 115, 2, 258, 6], [111, 259, 1, 260, 2, 9, 1

In [11]:
# Encode training data sentences into sequences
test_sequences = tokenizer.texts_to_sequences(testDocs)

# Pad the training sequences
test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=max_length)

# Output the results of our work
print("Test Doc Summary")
print("\nTest sequences:\n", test_sequences)
print("\nPadded test sequences:\n", test_padded[:5])
print("\nPadded test shape:", test_padded.shape)
print("Test sequences data type:", type(test_sequences))
print("Padded Test sequences data type:", type(test_padded))

Test Doc Summary

Test sequences:
 [[4, 30, 186, 187, 139], [4, 139], [4, 30, 911, 912, 103, 75, 913], [4, 914, 13], [4, 103, 75, 136], [4, 30, 103, 75, 136], [4, 30, 420, 421, 170, 350], [4, 30, 420, 421, 170, 76], [4, 73, 74, 422, 346], [4, 73, 74, 422, 13], [189, 206, 915], [343, 58, 184], [21, 59, 916, 21, 59, 917, 7], [918, 919, 327, 920], [423, 921, 26], [423, 70], [922, 923, 7], [924, 925, 926], [59, 142, 69, 927], [59, 142, 69, 104], [928, 929, 410, 930], [931, 424, 7], [424, 7], [932, 152, 26], [933, 934], [935, 936, 937, 938, 29], [72, 102, 29], [72, 102, 7], [102, 333], [939, 940, 29], [4, 140, 941], [4, 13], [4, 188, 138, 13], [425, 206, 942], [425, 206, 943], [944, 18, 25], [129, 25], [207, 426, 427], [], [207, 30, 945], [], [207, 311, 427], [], [391, 946, 392], [], [947, 948, 949, 950, 951, 952, 426, 953], [], [27, 954], [], [955, 956, 957, 428], [], [72, 27, 417, 344, 958, 428], [], [959], [], [960, 961, 419, 962], [], [963], [], [964, 965], [], [966, 967, 401], [], [429

In [12]:
#@title ENTER EPOCH
epochs =  100#@param {type:"integer"}

In [13]:
# define the model
model1 = Sequential()
model1.add(Dense(8, input_shape=(max_length,)))
#model.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
# compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model1.summary())
# fit the model
model1.fit(train_padded, trainLabels, epochs=epochs, verbose=0)
# evaluate the model
loss, accuracy = model1.evaluate(test_padded, testLabels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,353 (75.60 KB)

 Trainable params: 19,353 (75.60 KB)

 Non-trainable params: 0 (0.00 B)

None
Accuracy: 70.270270


Deep *NN*

In [14]:
input_dim = vocab_size+1
output_dim = 10

# define the model
model2 = Sequential()
model2.add(Embedding(input_dim, output_dim, input_length=max_length, name= 'embeded'))
model2.add(Flatten())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model2.summary())

# fit the model
model2.fit(train_padded, trainLabels, epochs=epochs, verbose=0)

# evaluate the model
loss, accuracy = model2.evaluate(test_padded, testLabels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embeded (Embedding)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Accuracy: 83.783782


Model with Word Embedding + **LSTM**


In [16]:
input_dim = vocab_size+1
output_dim = 10

# define the model
model3 = Sequential()
model3.add(Embedding(input_dim, output_dim, input_length=max_length, name= 'embeded'))
model3.add(SpatialDropout1D(0.25))
model3.add(LSTM(16, return_sequences=True))
model3.add(LSTM(8))
model3.add(Dropout(0.25))
model3.add(Dense(1, activation='sigmoid'))

# compile the model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model3.summary())

# fit the model
model3.fit(train_padded, trainLabels, epochs=epochs, verbose=0)

# evaluate the model
loss, accuracy = model3.evaluate(test_padded, testLabels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embeded (Embedding)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Accuracy: 78.378379


In [17]:
#@title Enter your statement
statement = "allah cezanı vermesin" #@param {type:"string"}

myTest=[statement]
myTestEncoded= tokenizer.texts_to_sequences(myTest)
print (myTestEncoded)
# Pad the training sequences
myTestPadded = pad_sequences(myTestEncoded, padding='post', truncating='post', maxlen=max_length)
print (myTestPadded)

print("Deep NN model ", 'Dua' if model1.predict(myTestPadded)[0][0]> 0.5 else 'Beddua', model1.predict(myTestPadded)[0][0])
print("Word Embedding ", 'Dua' if model2.predict(myTestPadded)[0][0]> 0.5 else 'Beddua', model2.predict(myTestPadded)[0][0])
print("Word Embedding + LSTM ", 'Dua' if model3.predict(myTestPadded)[0][0]> 0.5 else 'Beddua', model3.predict(myTestPadded)[0][0])


[[4, 126]]
[[  4 126   0   0   0   0   0   0   0   0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Deep NN model  Dua 0.6816389
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Word Embedding  Dua 0.99734586
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Word Embedding + LSTM  Dua 0.9995797


In [18]:
e= model3.get_layer(name='embeded')
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(976, 10)


In [19]:
import io
file_vec = 'vecs_'+str(epochs)+'.tsv'
file_meta= 'meta_'+str(epochs)+'.tsv'
out_v = io.open(file_vec, 'w', encoding='utf-8')
out_m = io.open(file_meta, 'w', encoding='utf-8')

for num, word in enumerate(tokenizer.word_index):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [20]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_vec)
  files.download(file_meta)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

http://projector.tensorflow.org/